In [1]:
import numpy as np 

In [2]:
# will return set of images of cardinality = num, each image has channels=3, width=3, height = 4
def get_set(num):
    set_ = []
    for i in xrange(num):
        image = []
        for j in xrange(3):
            channel = []
            for k in xrange(3):
                col = []
                for l in xrange(4):
                    col.append((i+1) * 1000 + (j+1) * 100 + (k+1) * 10 + (l+1))
                channel.append(col)
            channel = np.array(channel).transpose()
            image.append(channel)
        set_.append(image)
    set_=np.array(set_)
    return set_
set_ = get_set(8)

In [3]:
#mask shows how prepare convolution produce matrix of neighbourhood for one channel
mask = np.array([[00,00,00,00,11,12,00,21,22],
                 [00,00,00,11,12,13,21,22,23],
                 [00,00,00,12,13,14,22,23,24],
                 [00,00,00,13,14,00,23,24,00],
                 [00,11,12,00,21,22,00,31,32],
                 [11,12,13,21,22,23,31,32,33],
                 [12,13,14,22,23,24,32,33,34],
                 [13,14,00,23,24,00,33,34,00],
                 [00,21,22,00,31,32,00,00,00],
                 [21,22,23,31,32,33,00,00,00],
                 [22,23,24,32,33,34,00,00,00],
                 [23,24,00,33,34,00,00,00,00]])
#how channel look like
channel = [[11, 21, 31],
           [12, 22, 32],
           [13, 23, 33],
           [14, 24, 34]]
#result mask will count the occurances for each element of channel in prepared convolution matrix
result_mask = []
for row in channel:
    result_row = []
    for elem in row:
        result_row.append(np.sum((mask==elem).astype(np.int32)))
    result_mask.append(result_row)
result_mask = np.array(result_mask)

In [4]:
#this block prepare result for backpropogation of prepare convolution same
set_result = []
for image in set_:
    image_result = []
    for channel in image:
        image_result.append(np.multiply(channel, result_mask))
    set_result.append(image_result)
set_result = np.array(set_result)

    

In [5]:
# block will create input matrix of size (3*3*4, num) where 3*3*4 is size of each image and num is number of images 

def convert_channel_to_column(channel):
    return np.ravel(channel, order='F')

def convert_image_to_column(image):
    result = []
    for channel in image:
        result += convert_channel_to_column(channel).tolist()
    return result

def convert_set_to_matrix(set_):
    result = []
    for image in set_:
        result += convert_image_to_column(image)
    return np.reshape(np.array(result), [3*3*4, set_.shape[0]], order='F')
                       
output_matrix = convert_set_to_matrix(set_result)

In [6]:
np.savetxt('/home/ahsan/squirrel_latest/squirrel/trunk/src/Test/test-case-data/prepare-conv-same-back-r.txt', output_matrix, fmt='%i', delimiter=' ', header='36 8')